
# Alpha Earth – NDVI & Satellite Embeddings (GEE + leafmap)

Este notebook implementa un flujo **listo para ejecutar** con:
- **NDVI (Sentinel‑2)** para salud de vegetación.
- **Embeddings Satelitales** (`GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL`) para caracterización y análisis avanzado.
- **Visualización interactiva** con `leafmap/MapLibre`.

**Referencias:**
- Dataset: *Satellite Embedding V1 (Annual)* – `GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL`
- Tutoriales GEE (Embeddings): Introducción, Clasificación, Regresión, Búsqueda de similitud
- Mapas web: `leafmap` / `maplibre`

> Sugerencia: si usas **Colab**, ejecuta todas las celdas en orden. Si usas Jupyter local, asegúrate de tener las credenciales de Earth Engine configuradas.


In [2]:

import sys

def _pip(cmd):
    print(f"Running: {cmd}")
    !{sys.executable} -m pip install --quiet --upgrade {cmd}

_pkgs = [
    "earthengine-api",
    "geemap>=0.32.0",
    "leafmap>=0.34.0"
]

for p in _pkgs:
    _pip(p)

print("✅ Dependencias instaladas/actualizadas.")


Running: earthengine-api
Running: geemap>=0.32.0



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\USUARIO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Running: leafmap>=0.34.0



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\USUARIO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


✅ Dependencias instaladas/actualizadas.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\USUARIO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:

# === Autenticación e inicialización de Earth Engine ===
import ee, os

try:
    ee.Initialize()
    print("✅ Earth Engine inicializado (credenciales encontradas).")
except Exception as e:
    print("ℹ️ Autenticación requerida. Si estás en Colab, aparecerá una ventana emergente.")
    ee.Authenticate()
    ee.Initialize()
    print("✅ Earth Engine autenticado e inicializado.")


ℹ️ Autenticación requerida. Si estás en Colab, aparecerá una ventana emergente.


In [ ]:

# === Imports generales y utilidades ===
import datetime as dt
import ee
import leafmap

# Función de enmascaramiento de nubes para Sentinel-2 SR
def mask_s2_sr(image):
    # QA60: bits 10 y 11 son nubes y cirros
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)                .select(['B2','B3','B4','B8','B11','B12'])                .copyProperties(image, image.propertyNames())


In [ ]:

# === Parámetros del análisis ===
YEAR = 2025
START = ee.Date.fromYMD(YEAR, 7, 1)   # Ajusta el rango temporal a tu necesidad
END   = ee.Date.fromYMD(YEAR, 7, 31)
CLOUDY_PCT = 20  # porcentaje máximo de nubes permitido

# === Área de interés (AOI) ===
# Ejemplo: un punto en Colombia y un buffer de 2 km
aoi_point = ee.Geometry.Point([-75.514, 6.251])  # (lon, lat) – Medellín aprox.
AOI = aoi_point.buffer(2000)  # 2km
print("AOI listo.")



## NDVI con Sentinel‑2 (SR)

Pasos:
1. Filtrar colección `COPERNICUS/S2_SR` por AOI, fechas y nubes.
2. Enmascarar nubes usando `QA60`.
3. Componer una imagen (mediana) y calcular **NDVI**.
4. Visualizar en `leafmap` con un *heatmap* rojo→amarillo→verde.


In [ ]:

# === Cargar y filtrar Sentinel-2 SR ===
s2_sr = (ee.ImageCollection("COPERNICUS/S2_SR")
         .filterBounds(AOI)
         .filterDate(START, END)
         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUDY_PCT))
         .map(mask_s2_sr))

s2_median = s2_sr.median()

# === NDVI ===
ndvi = s2_median.normalizedDifference(['B8','B4']).rename('NDVI')

# === Visualización NDVI ===
ndvi_vis = {
    'min': -0.2,
    'max':  0.9,
    'palette': ['#a50026', '#f46d43', '#fee08b', '#a6d96a', '#1a9850']  # rojo -> amarillo -> verde
}

true_color_vis = {'bands': ['B4','B3','B2'], 'min': 0.0, 'max': 0.3}

m = leafmap.Map(center=[6.251, -75.514], zoom=12, tiles="CartoDB.Positron")
m.add_basemap("Esri.WorldImagery")
m.addLayer(s2_median, true_color_vis, "Sentinel-2 True Color")
m.addLayer(ndvi, ndvi_vis, "NDVI (Sentinel-2)")
m.addLayer(AOI, {}, "AOI")
m.add_colorbar(colors=ndvi_vis['palette'], vmin=ndvi_vis['min'], vmax=ndvi_vis['max'], label="NDVI")
m


In [ ]:

# === (Opcional) Exportar NDVI a Google Drive ===
# Descomenta para exportar (necesita permisos de Drive vinculados a tu cuenta EE)
# task = ee.batch.Export.image.toDrive(
#     image=ndvi.clip(AOI),
#     description=f"NDVI_{YEAR}_Jul",
#     folder="AlphaEarth_Exports",
#     fileNamePrefix=f"NDVI_{YEAR}_Jul",
#     scale=10,
#     region=AOI,
#     maxPixels=1e13
# )
# task.start()
# print("Export a Drive iniciado:", task.id)



## Embeddings Satelitales (GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL)

- Cada píxel contiene un **vector de 64 dimensiones** con información anual multisensor.
- Flujo:
  1. Cargar la colección y filtrar por año y AOI.
  2. Hacer *mosaic* (una sola imagen por año/ROI).
  3. **Samplear** vectores en la AOI (para explorar o entrenar modelos).
  4. (Opcional) Visualización PCA RGB de los embeddings.


In [ ]:

# === Carga de Embeddings (Anual) ===
emb_ic = (ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
          .filterDate(ee.Date.fromYMD(YEAR,1,1), ee.Date.fromYMD(YEAR+1,1,1))
          .filterBounds(AOI))

emb_img = emb_ic.mosaic()  # imagen única
band_names = emb_img.bandNames()
print("Número de bandas (dimensión del embedding):", band_names.size().getInfo())
print("Primeras bandas:", band_names.slice(0,5).getInfo())

# === Muestreo de vectores (64-dim) dentro del AOI ===
samples = emb_img.sample(region=AOI, scale=10, numPixels=200, geometries=True)
print("Muestras obtenidas:", samples.size().getInfo())

# Ver una muestra
samp_first = samples.first()
print("Ejemplo de vector (primer feature):")
print(samp_first.toDictionary().getInfo())


In [ ]:

# === PCA (3 componentes) para visualización RGB de los embeddings ===
# Convierte bandas a array para aplicar PCA
array_img = emb_img.toArray()
# Reducer PCA retornando 3 componentes
pca = array_img.reduce(ee.Reducer.principalComponents(3))
# La salida es un array; convertimos a bandas separadas
# Dimensión 0: pixel, Dimensión 1: componente
pca_bands = pca.arrayProject([0]).arrayFlatten([['PC1', 'PC2', 'PC3']])

# Stretch sencillo para visualización
pca_vis = {'min': -2, 'max': 2}
m2 = leafmap.Map(center=[6.251, -75.514], zoom=12, tiles="CartoDB.Positron")
m2.addLayer(pca_bands, pca_vis, f'Embeddings PCA RGB {YEAR}')
m2.addLayer(AOI, {}, "AOI")
m2


In [ ]:

# === (Opcional) Exportar muestras a Drive como tabla CSV ===
# Descomenta para exportar (requiere permisos de Drive)
# task_tbl = ee.batch.Export.table.toDrive(
#     collection=samples,
#     description=f"Embeddings_Samples_{YEAR}",
#     folder="AlphaEarth_Exports",
#     fileNamePrefix=f"embeddings_samples_{YEAR}",
#     fileFormat='CSV'
# )
# task_tbl.start()
# print("Export de muestras a Drive iniciado:", task_tbl.id)



## ¿Cómo interviene *Alpha Earth* en el flujo?

- **Ingesta (GEE → Notebook):** este notebook automatiza la descarga/procesado de **NDVI** y **Embeddings** para el AOI del productor.
- **Features & ML:** los vectores de **64‑D** se combinan con índices (NDVI/NDWI/LST) y datos IoT para *clustering*, clasificación o regresión (rendimiento, estrés hídrico).
- **Visualización Web:** los *rasters* (NDVI/PCA) se publican con `leafmap/maplibre` (o exportados como tiles/GeoTIFF) para el **dashboard** de Alpha Earth.
- **Alertas:** con umbrales (p.ej., NDVI < 0.3) se generan **semáforos** y notificaciones para decisiones agronómicas.
